# Consulta no banco do Corpore RM e comparação de dados na FLAN
Com posse de uma planilha de lançamentos, descobriremos quais foram baixados, os respectivos valores e também se houveram lançamentos deletados.

É o básico de extração de dados.


In [ ]:
# Importação do Pandas e drive ODBC
import pandas as pd
import pyodbc

In [ ]:
# leitura da planilha. A mesma deve estar na raiz.
df = pd.read_excel('dados.xlsx')

In [ ]:
# Seleção dos dados da coluna
coluna = df["IDLAN"].tolist()

In [ ]:
# Estabelecer a conexão com SQL Server
server = 'servername'
database = 'CorporeRM'
username = 'user'
password = 'password'
conexao = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+server+';'
    'DATABASE='+database+';'
    'UID='+username+';'
    'PWD='+password)

In [ ]:
# Converter o array de inteiros em string com aspas e separada por vírgula. A função split não funciona com inteiros.
lista = ''
for num in coluna:
    lista += str(num) + ','
lista = lista[:-1] # remove a vírgula que sobrou

# Consulta SQL
query = ('''select FLAN.CODCOLIGADA, FCFO.NOME, IDLAN, DESCRICAO, HISTORICO, CAST(VALORORIGINAL AS MONEY ) AS VALORORIGINAL, 
CAST(VALORBAIXADO AS MONEY) AS VALORBAIXADO, DATABAIXA from flan INNER JOIN FCFO ON FLAN.CODCFO = FCFO.CODCFO AND FLAN.CODCOLCFO = FCFO.CODCOLIGADA left join ZMDSTATUSLAN ON FLAN.STATUSLAN = ZMDSTATUSLAN.ID
where IDLAN in ({}) and FLAN.CODCOLIGADA = '1' ''').format(lista)

In [ ]:
# Consulta no banco e converte o resultado em Excel.
df2 = pd.read_sql_query(query, conexao)
df2.to_excel("Planilha.xlsx")
df2.head(10)

In [ ]:
#Comparação das duas planilhas e saída em excel.
df_all = pd.concat([df.set_index('IDLAN'), df2.set_index('IDLAN')], 
                   axis='columns', keys=['ORIGINAL','CONSULTA'])
df_all.to_excel("Planilha_Comparacao.xlsx")